In [1]:
from nltk.tokenize import sent_tokenize
from gensim.models import Word2Vec
import nltk
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
import glob
import os
from os import path
import string
import numpy
import re
from __future__ import absolute_import, division, print_function
import pandas as pd
import multiprocessing
import gensim
import random
from tqdm import tqdm
import pandas as pd
from gensim.models import KeyedVectors
from numpy import array
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
os.chdir("C://Users//Ruben//Documents//GitHub//TheForeign//SCC//output-data")
tt = pd.read_csv('tokens_year.csv')

In [33]:
os.chdir("D://Scriptie//Data//models")
list_mod = glob.glob('*w2v')

In [49]:
def GetNetwork(word, year, similarity_cutoff, freq_threshold, n):
    mfn = [m for m in list_mod if int(m[0:4]) == year]
    
    #model = KeyedVectors.load_word2vec_format(mfn, binary = True)
    model = KeyedVectors.load(mfn[0], mmap='r')
    
    
    ttp = tt[(tt.year >= year) & (tt.year < (year + 10))]
    ttp = ttp['tokens'].sum()
    
    dft = pd.DataFrame()
    
    for ms in list(dict(model.most_similar(word, topn=n)).keys()):
        
        level_1_list = list(dict(model.most_similar(ms, topn=n)).keys())
        
        level_1_df = pd.DataFrame()
        
        for ms2 in level_1_list:
            level_2_list = list(dict(model.most_similar(ms2, topn=n)).keys())

            level2_df = pd.DataFrame()

            for term in level_2_list:
                tmp_df = pd.DataFrame([[key,value] for key,value in dict(model.most_similar(term, topn=n)).items()], columns = ['target', 'weight'])
                tmp_df['source'] = term
                level2_df = level2_df.append(tmp_df)

            level_1_df = level_1_df.append(level2_df)
            
        dft = dft.append(level_1_df)
    
    dft['count'] = [model.wv.vocab[w].count for w in list(dft.target)]
    dft['count'] = dft['count'] / ttp
    
    dft = dft[dft['count'] > freq_threshold]
    dft = dft[dft.weight > similarity_cutoff]
    
    dgc = Counter(list(dft.target))
    dgc = [k for k,v in dict(dgc).items() if v >= 10]
    
    dft['degree'] = 1
    
    dft = dft[dft.target.isin(dgc)]
    
    return dft

In [60]:
os.chdir("D://Scriptie//Data//models")
df = pd.DataFrame()

for i in range(1875,1915,10):
    x = GetNetwork("verre_landen", i,0.3,0,15)
    x['year'] = i
    df = df.append(x)
    
os.chdir('C://Users//Ruben//Desktop')
df.to_csv('verre_landen_dia.csv', index=False)



C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  del sys.path[0]
C:\ProgramData\Anaconda3\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  from ipykernel import kernelapp as app
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: Deprec